# Hydrostatic Equilibrium Solver

In [7]:
using Pkg
Pkg.activate("..")

using GLMakie
using NonRelativisticMergers
using LinearAlgebra
using LinearSolve
using NonlinearSolve
using OrdinaryDiffEq

import NonRelativisticMergers: solve_poisson

  Activating project at `~/gitrep/NonRelativisticMergers`


**Goal:** Solve the hydrostatic equation
$$
\nabla P = - \rho \nabla \phi \; ,
$$
numerically. This will be needed to obtain the initial condition for the merger simulation.

We start from some density distribution $\rho(x)$ and find the gravitational potential $\phi(x)$ by solving the Poisson equation
$$
\nabla^2 \phi = 4\pi G \rho \; .
$$

### 1d case

In 1d we have to solve
$$
\partial_x P(x) = - \rho(x) \partial_x \phi(x) \; .
$$

We could solve this by direct integration, but this will not be possible for the 2d case. Instead we can use finite difference and a non-linear solver to find the expression of $P(x)$. 

In [33]:
(; xl, Nx) = gd = Grid1D(; xmin=-1, xmax=1, Nx=500)
prob = FDProblem(gd, EulerSelfGravity(), KT(), HLLC())

ρ = [0.1 + 1.8 * exp(-100 * x^2) for x in xl]
ϕ = solve_poisson(prob, ρ);

In [34]:
∇x = Matrix(Tridiagonal(ones(Nx - 1), zeros(Nx), -ones(Nx - 1)))
∇x[1, Nx] = 1
∇x[Nx, 1] = -1

∇ϕ = ∇x * ϕ;

In [37]:
nlprob = NonlinearProblem((P, _) -> ∇x * P .+ ρ .* ∇ϕ, zeros(Nx))
nlsol = NonlinearSolve.solve(nlprob)
P = nlsol.u

500-element Vector{Float64}:
 -0.006379837479297162
 -0.006379833798117487
 -0.006379826429842946
 -0.006379815362636525
 -0.006379800578716267
 -0.006379782054330686
 -0.006379759759717205
 -0.006379733659057125
 -0.006379703710415471
 -0.006379669865676651
  ⋮
 -0.00637970371041575
 -0.006379733659056898
 -0.006379759759717407
 -0.006379782054330513
 -0.006379800578716566
 -0.006379815362636275
 -0.006379826429843247
 -0.006379833798117204
 -0.00637983747929745

In [35]:
lalg = SVDFactorization()
# lalg = QRFactorization()

lprob = LinearProblem(∇x, -ρ .* ∇ϕ)
lsol = LinearSolve.solve(lprob, lalg)
P = lsol.u

500-element Vector{Float64}:
 -0.006372686045867099
 -0.007320575341375153
 -0.0063726749964128845
 -0.007320556905894192
 -0.006372649145286201
 -0.007320523597588339
 -0.006372608326287138
 -0.00732047520231479
 -0.00637255227698541
 -0.007320411408934319
  ⋮
 -0.0073204452536734105
 -0.006372582225626835
 -0.0073205013029750705
 -0.006372630620900453
 -0.007320542121974229
 -0.006372663929206213
 -0.007320567973100909
 -0.006372682364687143
 -0.007320579022555118

In [38]:
f = lines(ρ, label="rho")
lines!(ϕ, label="phi")
lines!(P, label="P")
axislegend()
f

### 2d case

In the 2d case we can consider a spherically symmetric distribution for the pressure, i.e. $P = P(r)$ with $r=\sqrt{x^2 + y^2}$.

We can combine the Poisson and hydrostatic equilibrium equation to obtain
$$
\nabla\left( \frac{\nabla P}{\rho} \right) + 4\pi G \rho = 0 \; ,
$$
which becomes 
$$
\rho \partial_r^2 p 
+ \frac{\rho}{r}\partial_r p 
- (\partial_r\rho)(\partial_r p) 
+ 4\pi G \rho^3 = 0 \; .
$$
when $p=p(r)$ and $\rho=\rho(r)$.

We can then assume a perfect gas equation of state
$$
p = K\rho^\gamma \; ,
$$
to obtain
$$
\partial_r^2 \rho 
+ \frac{1}{r} \partial_r \rho 
+ \frac{\gamma - 2}{\rho}(\partial_r \rho)^2
+ \frac{4\pi G}{K\gamma} \rho^{3-\gamma} = 0 \; .
$$

In [57]:
γ = 5 / 3
K = 1
G = 1

ρ0 = 10
r0 = 1e-5
rmax = 2

function rhs!(du, u, _, r)
    ρ, dρdr = u

    try
        du[1] = dρdr
        du[2] = -1 / r * dρdr - (γ - 2) / ρ * dρdr^2 - 4π * G / (K * γ) * ρ^(3 - γ)
    catch e
        if e isa DomainError
            du[1] = NaN
            du[2] = NaN
        else
            rethrow(e)
        end
    end
end

prob = ODEProblem(rhs!, [ρ0, 0], (r0, rmax))
hs_sol = OrdinaryDiffEq.solve(prob, Tsit5())

retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 90-element Vector{Float64}:
 1.0e-5
 1.6155482840629428e-5
 2.0838620046084053e-5
 3.023266280851059e-5
 4.15403288220415e-5
 6.016481979819523e-5
 8.689097133163797e-5
 0.00012992583381439503
 0.00019776007320502709
 0.00031114010304899334
 ⋮
 0.8551770203242546
 0.8802335660819414
 0.9179663519207248
 0.9758983170436681
 1.0686971655924549
 1.2179655111067584
 1.4255496498879934
 1.7436607569339286
 2.0
u: 90-element Vector{Vector{Float64}}:
 [10.0, 0.0]
 [9.99999999735752, -0.0008094115108448318]
 [9.999999992389323, -0.001302760025369999]
 [9.99999997592826, -0.0021868544166745245]
 [9.999999945550377, -0.003178395388602423]
 [9.99999987163494, -0.004751606830870826]
 [9.999999715012896, -0.006963834588329887]
 [9.999999339360171, -0.010490098497065808]
 [9.999998440076867, -0.016021055221875885]
 [9.999996100589016, -0.02524477226482378]
 ⋮
 [9.921072582223623e-6, 0.00028473102109748344]
 [1.772686931407

In [52]:
lines(r0:0.01:rmax, r -> sol(r)[1], label="rho")

We can check that the solution is indeed hydrostatic by using it as a initial condition.

In [66]:
gd = Grid2D(; xmin=-4, xmax=4, ymin=-4, ymax=4, Nx=2^7, Ny=2^7)

ρ0 = [hs_sol(sqrt(x^2 + y^2))[1] < 0.01 ? 0.01 : hs_sol(sqrt(x^2 + y^2))[1] for x in gd.xl, y in gd.yl]
vx0 = zeros(gd.Nx, gd.Ny)
vy0 = zeros(gd.Nx, gd.Ny)
P0 = @. K * ρ0^γ

prob = FDProblem(gd, EulerSelfGravity(; γ, G, ϵ=0), KT(), HLLC())

sol = NonRelativisticMergers.solve(prob, ρ0, vx0, vy0, P0, (0, 2))

retcode: Success
Interpolation: 1st order linear
t: 100-element Vector{Float64}:
 0.0
 0.020202020202020204
 0.04040404040404041
 0.06060606060606061
 0.08080808080808081
 0.10101010101010101
 0.12121212121212122
 0.1414141414141414
 0.16161616161616163
 0.18181818181818182
 ⋮
 1.8383838383838385
 1.8585858585858586
 1.878787878787879
 1.898989898989899
 1.9191919191919191
 1.9393939393939394
 1.9595959595959596
 1.97979797979798
 2.0
u: 100-element Vector{Array{Float64, 3}}:
 [0.01 0.01 … 0.01 0.01; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0006962383250419165 0.0006962383250419165 … 0.0006962383250419165 0.0006962383250419165;;; 0.01 0.01 … 0.01 0.01; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0006962383250419165 0.0006962383250419165 … 0.0006962383250419165 0.0006962383250419165;;; 0.01 0.01 … 0.01 0.01; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0006962383250419165 0.0006962383250419165 … 0.0006962383250419165 0.0006962383250419165;;; … ;;; 0.01 0.01 … 0.01 0.01; 0.0 0.0 … 0.0 0.0; 0.0 0.0

In [67]:
NonRelativisticMergers.plot_euler(prob, sol; type=:surface)
# NonRelativisticMergers.plot_euler2(prob, sol)